In [1]:
import tensorflow as tf
import keras
from keras import backend as K
import matplotlib.pyplot as plt
import numpy as np
import glob
import cv2
from sklearn.model_selection import train_test_split
import re
import os
import time
from nn_helpers import get_labeled_images, freeze_session, load_graph, one_hot_encode

print(tf.__version__)

Using TensorFlow backend.


1.3.0


In [2]:
X_train, X_test, y_train, y_test = get_labeled_images('dataset_site/all/*.jpg')
one_hot_test = one_hot_encode(y_test)
one_hot_train = one_hot_encode(y_train)
X_train = np.array(X_train)
X_test = np.array(X_test)
    
print(X_train.shape, one_hot_train.shape)
print(X_test.shape, one_hot_test.shape)

img = np.expand_dims(X_test[0],0)

((181, 75, 100, 3), (181, 3))
((46, 75, 100, 3), (46, 3))


In [3]:
K.set_learning_phase(0)
K.clear_session()
model = keras.models.load_model('./carla_traffic_light_classifier.h5')
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 75, 100, 16)       448       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 37, 50, 16)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 37, 50, 32)        4640      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 18, 25, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 18, 25, 64)        18496     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 9, 12, 64)         0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 9, 12, 64)         0         
__________

In [4]:
start = time.time()
predictions = model.predict(img)
print("%.2gs" % (time.time() - start))
print("Predictions: {}".format(np.argmax(predictions[0])))

0.013s
Predictions: 0


In [5]:
model.evaluate(X_test, one_hot_test)

32/46 [===================>..........] - ETA: 0s

[0.0076310647773030014, 1.0]

In [6]:
frozen_graph = freeze_session(K.get_session(), output_names=[out.op.name for out in model.outputs])
tf.train.write_graph(frozen_graph, './', 'carla_traffic_light_classifier.pbtxt', as_text=True)
tf.train.write_graph(frozen_graph, './', 'carla_traffic_light_classifier.pb', as_text=False)

INFO:tensorflow:Froze 35 variables.
Converted 35 variables to const ops.


'./carla_traffic_light_classifier.pb'

In [7]:
graph = load_graph('carla_traffic_light_classifier.pb')

In [8]:
for op in graph.get_operations():
    print(op.name)

prefix/keras_learning_phase
prefix/conv2d_1_input
prefix/conv2d_1/kernel
prefix/conv2d_1/kernel/read
prefix/conv2d_1/bias
prefix/conv2d_1/bias/read
prefix/conv2d_1/convolution
prefix/conv2d_1/BiasAdd
prefix/conv2d_1/Relu
prefix/max_pooling2d_1/MaxPool
prefix/conv2d_2/kernel
prefix/conv2d_2/kernel/read
prefix/conv2d_2/bias
prefix/conv2d_2/bias/read
prefix/conv2d_2/convolution
prefix/conv2d_2/BiasAdd
prefix/conv2d_2/Relu
prefix/max_pooling2d_2/MaxPool
prefix/conv2d_3/kernel
prefix/conv2d_3/kernel/read
prefix/conv2d_3/bias
prefix/conv2d_3/bias/read
prefix/conv2d_3/convolution
prefix/conv2d_3/BiasAdd
prefix/conv2d_3/Relu
prefix/max_pooling2d_3/MaxPool
prefix/dropout_1/cond/Switch
prefix/dropout_1/cond/switch_t
prefix/dropout_1/cond/pred_id
prefix/dropout_1/cond/mul/y
prefix/dropout_1/cond/mul/Switch
prefix/dropout_1/cond/mul
prefix/dropout_1/cond/dropout/keep_prob
prefix/dropout_1/cond/dropout/Shape
prefix/dropout_1/cond/dropout/random_uniform/min
prefix/dropout_1/cond/dropout/random_unifo

In [9]:
print(graph)

In [10]:
# We access the input and output nodes 
x = graph.get_tensor_by_name('prefix/conv2d_1_input:0')
y = graph.get_tensor_by_name('prefix/dense_2/Softmax:0')
lf = graph.get_tensor_by_name('prefix/keras_learning_phase:0')
img = np.expand_dims(X_test[0],0)

In [11]:
with tf.Session(graph=graph) as sess:
    y_out = sess.run(y, feed_dict={x: img, lf: 0})
    
    print(np.argmax(y_out), y_test[0])

(0, 0)


In [12]:
# from tensorflow.python.framework.graph_util import load_graph